In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
import json
import inspect

client = OpenAI()

def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

def vs_to_schema(id):
    return {
        "type": "file_search",
        "file_search": {
            "vector_store_ids": [id],
        },
    }


class Agent(BaseModel):
    name: str = "Agent"
    model: str = "gpt-4o-mini"
    instructions: str = "You are a helpful Agent"
    tools: list = []
    vs_id: str = ""
    
class Response(BaseModel):
    agent: Optional[Agent]
    messages: list

def run_full_turn(agent, messages):

    current_agent = agent
    num_init_messages = len(messages)
    messages = messages.copy()
    
    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in current_agent.tools]
        tools = {tool.__name__: tool for tool in current_agent.tools}

        # add vector_store to "File_Search" tool
        # if current_agent.vs_id : tool_schemas.append(vs_to_schema(current_agent.vs_id))   
        # print("tool_schemas", tool_schemas)
        
        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.model,
            messages=[{"role": "system", "content": current_agent.instructions}]
            + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print agent response
            print(f"\033[31m{current_agent.name}:\033[0m", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools, current_agent.name)

            if type(result) is Agent:  # if agent transfer, update current agent
                current_agent = result
                result = (
                    f"Transfered to {current_agent.name}. Adopt persona immediately."
                )

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

    # ==== 3. return last agent used and new messages =====
    return Response(agent=current_agent, messages=messages[num_init_messages:])


def execute_tool_call(tool_call, tools, agent_name):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"\033[31m{agent_name}:", f"\033[32m{name}({args})\033[0m")

    return tools[name](**args)  # call corresponding function with provided arguments

In [ ]:
from sec_api import QueryApi
from sec_api import ExtractorApi
from sec_api import XbrlApi
from sec_api import SubsidiaryApi
from sec_api import ExecCompApi
from IPython.display import display, HTML

import pandas as pd
import json

SEC_API_KEY = "af874bc49e22a0856ed80ec4c3f46b7e3b3a5891112931777181f3f0e7c7ca18"
queryApi = QueryApi(api_key= SEC_API_KEY)
extractorApi = ExtractorApi(SEC_API_KEY)
xbrlApi = XbrlApi(SEC_API_KEY)
subsidiary_api = SubsidiaryApi(SEC_API_KEY)
execCompApi = ExecCompApi(SEC_API_KEY)

def extract_items_10k(filing_url,items,ex_type="text"):

    for item in items:
        print(f"Extracting item {item} from 10-K filing {filing_url}")

        try:
            section_text = extractorApi.get_section(
                filing_url=filing_url, section=item, return_type=ex_type
            )
            if ex_type == "html":
                display(HTML(section_text))
            else:
                print(f"Section item {item} : {section_text}")
            # Process section_text as needed: save to disk, store in a database, or perform analytics.
            # IMPORTANT: Avoid holding a large number of sections in memory by appending them to a list,
            # as this can lead to out-of-memory issues. Instead, ensure that memory is freed regularly
            # by allowing garbage collection to manage unused objects.

        except Exception as e:
            print(e)


def get_filing_items(company_name,items,ex_type="text"):
    start_date = '2024-01-01'
    end_date = '2024-12-31'
    form = '"10-K"'
    search_query = 'companyName:'+company_name +' AND formType:'+form+' AND filedAt:['+ start_date+' TO '+end_date+']'
    parameters = {
        "query": search_query,
        "from": "0",
        "size": "1",
        "sort": [{"filedAt": {"order": "desc"}}],
    }

    response = queryApi.get_filings(parameters)
   
    # for each filing, get the URL of the filing
    # set in the dict key "linkToFilingDetails"
    urls_list = list(
        map(lambda x: x["linkToFilingDetails"], response["filings"])
    )

    # get the standardized and cleaned text of section 1A "Risk Factors"
    
    for filing_url in urls_list :
        extract_items_10k(filing_url,items,ex_type)


def get_filing_financials(company_name, table='BalanceSheets'):
    start_date = '2024-01-01'
    end_date = '2024-12-31'
    form = '"10-K" AND dataFiles.description:"XBRL INSTANCE DOCUMENT"'
    
    search_query = 'companyName:'+company_name +' AND formType:'+form+' AND filedAt:['+ start_date+' TO '+end_date+']'
    parameters = {
        "query": search_query,
        "from": "0",
        "size": "1",
        "sort": [{"filedAt": {"order": "desc"}}],
    }

    response = queryApi.get_filings(parameters)
   
    # for each filing, get the URL of the filing
    # set in the dict key "linkToFilingDetails"
    urls_list = list(
        map(lambda x: x["linkToFilingDetails"], response["filings"])
    )

    # get the standardized and cleaned text of section 1A "Risk Factors"    
    #table = 'StatementsOfIncome'
    #table = 'BalanceSheets'
    #table = 'StatementsOfCashFlows'
    xbrl_json = [xbrlApi.xbrl_to_json(filing_url)[table] for filing_url in urls_list ]
    return xbrl_json

def get_subsidaries(company_name):
    start_date = '2017-01-01'
    end_date = '2024-12-31'

    search_query = 'companyName:'+company_name #+ ' AND fileAt:['+ start_date+' TO '+end_date+']'
    parameters = {
      "query": search_query,
      "from": "0",
      "size": "50",
      "sort": [ { "filedAt": { "order": "desc"  } } ]
    }
    list_subsidiaries = subsidiary_api.get_data(parameters)
    #print("Most recent subsidiary data:")
    #list_subsidiaries['data'][0]
    subsidiaries_df = pd.DataFrame(list_subsidiaries['data'])
    # drop columns: id, companyName
    subsidiaries_df = subsidiaries_df.drop(['id', 'companyName'], axis=1)
    # explode subsidiaries column
    subsidiaries_df = subsidiaries_df.explode('subsidiaries')
    # explode subsidiaries column to new "name" and "jurisdiction" columns
    subsidiaries_df[['name', 'jurisdiction']] = subsidiaries_df['subsidiaries'].apply(pd.Series)
    # drop subsidiaries column
    subsidiaries_df = subsidiaries_df.drop(['subsidiaries'], axis=1)
    print("All "+company_name+" subsidiaries:")
    print(subsidiaries_df)
    #print("Unique reporting dates:", subsidiaries_df['filedAt'].nunique(), "\n", subsidiaries_df['filedAt'].unique())
    print("subsidary names:", subsidiaries_df['name'].nunique(), "\n", subsidiaries_df['name'].sort_values().unique())
    print("juristdictions:", subsidiaries_df['jurisdiction'].nunique(), "\n", subsidiaries_df['jurisdiction'].sort_values().unique())
    return "success"

def get_exec_comp(company_name):
    
    parameters = {
        "query": 'companyName:'+company_name ,
        "from": "0",
        "size": "1",
        "sort": [{"filedAt": {"order": "desc"}}],
    }

    ticker_name = queryApi.get_filings(parameters)['filings'][0]['ticker']

    search_query = 'ticker:'+ ticker_name+ ' AND year:[2022 TO 2023]'
    query = {
        "query": search_query,
        "from": "0",
        "size": "50",
        "sort": [{"total": {"order": "desc"}}]
    }

    compensation = execCompApi.get_data(query)
    df = pd.DataFrame(compensation)
    print(df.head())    

    return "success"


def escalate_to_human(summary):
    """Only call this if explicitly asked to."""
    print("Escalating to human agent...")
    print("\n=== Escalation Report ===")
    print(f"Summary: {summary}")
    print("=========================\n")
    exit()


def transfer_to_equity_analyst():
    """User for equity related questions."""
    return equity_analyst


def transfer_to_fixedIncome_analyst():
    """User for fixed income realted questions."""
    return fixed_income_analyst


def transfer_back_to_triage():
    """Call this if the user brings up a topic outside of your purview,
    including escalating to human."""
    return triage_agent

def transfer_to_macro_economist():
    """User for Macro Economic or Geopolitical questions."""
    return economic_analyst

def transfer_to_tax_specialist():
    """USer for any tax related questions."""
    return tax_analyst


triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You are a Investment Advisor for Blue Hills Research. "
        "Introduce yourself. Always be very brief. "
        "Gather information to direct the customer to the right analyst. "
        "But make your questions subtle and natural."
    ),
    tools=[transfer_to_equity_analyst, transfer_to_fixedIncome_analyst, transfer_to_macro_economist, transfer_to_tax_specialist, escalate_to_human],
)


def study_business (company):
    """Use to study further business quality 
    company can be a name of company or Ticker."""

    print("\033[32mStudy business:\033[0m", company) # lazy summary

    #get_filing_items(company,items=["1"])

    get_filing_items(company,items=["1","7","8"], ex_type="html")

    return "success"


def study_management (company):
    """Use to study further management quality 
    company can be a name of company or Ticker."""

    print("\033[32mStudy management:\033[0m", company) # lazy summary
    get_filing_items(company,items=["10","11"])

    return "success"

def study_risks (company):
    """Use to study Risk factors associated with the company  
    company can be a name of company or Ticker."""

    print("\033[32mStudy Risk Factors:\033[0m", company) # lazy summary
    get_filing_items(company,items=["1A"], ex_type="html")

    return "success"

def study_capital_allocation (company):
    """Use to study further capital allocation quality 
    company can be a name of company or Ticker."""

    print("\033[32mStudy capital allocation:\033[0m", company) # lazy summary
    get_filing_items(company,items=["8"], ex_type="html")
    
    return "success"

def get_balance_sheet(company):
    """Use to get a company's recent year balance sheet 
    company can be a name of company or Ticker."""

    print("\033[32mBalance Sheet :\033[0m", company) # lazy summary
    
    list_financials = get_filing_financials(company,table='BalanceSheets')
    # Convert JSON to a DataFrame
    df = pd.DataFrame(list_financials)
    #df.to_csv("output.csv", index=False)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    print(df)
    pd.reset_option('display.max_columns')
    pd.reset_option('display.max_rows')

    return "success"

def get_shareholders_equity(company):
    """Use to get a company's statement of shareholder's equity 
    company can be a name of company or Ticker."""

    print("\033[32mStatement of shareholders equity :\033[0m", company) # lazy summary
    
    list_financials = get_filing_financials(company,table='StatementsOfShareholdersEquity')
    # Convert JSON to a DataFrame
    df = pd.DataFrame(list_financials)
    pd.set_option('display.max_columns', None)
    print(df)
    pd.reset_option('display.max_columns')
    return "success"


def get_income_statement(company):
    """Use to get a company's recent year Income Statement 
    company can be a name of company or Ticker."""

    print("\033[32mIncome Statement :\033[0m", company) # lazy summary
    
    list_financials = get_filing_financials(company,table='StatementsOfIncome')
    # Convert JSON to a DataFrame
    df = pd.DataFrame(list_financials)
    pd.set_option('display.max_columns', None)
    print(df)
    pd.reset_option('display.max_columns')
    return "success"

def get_cashflow_statement(company):
    """Use to get a company's recent year Cashflow Statement 
    company can be a name of company or Ticker."""

    print("\033[32mCashflow Statement :\033[0m", company) # lazy summary
    
    list_financials = get_filing_financials(company,table='StatementsOfCashFlows')
    # Convert JSON to a DataFrame
    df = pd.DataFrame(list_financials)
    pd.set_option('display.max_columns', None)
    print(df)
    pd.reset_option('display.max_columns')
    return "success"


equity_analyst = Agent(
    name="Equity Analyst",
    instructions=(
        "You are an Equity Analyst from Blue Hills Research."
        "Your client is a fund portfolio manager who rely on your deep knowledge on indivudal company and industry sector."
        "You should help him analyse a company, and you should ways carry analytical and critical reasoning."
        "Follow the following routine with the client:"
        "1. First, ask probing questions and understand the what he really like to understand.\n"
        " - unless the he has already provided a reason.\n"
        "2. based on his request, you should decide further to invoke studies of business qualities,\n"
        "management quality, risks, company subsidaries, company capital allocation strategy,\n"
        "Or simply get company's financials like Balance Sheets, Income Statements, Cashflows statements,\n"
        " Statement of Shareholders equity, exective compensations.\n"
        ""
    ),
    tools=[
        study_business,study_management,study_capital_allocation, study_risks, get_subsidaries, 
        get_balance_sheet, get_income_statement, get_cashflow_statement, get_shareholders_equity,
        get_exec_comp, transfer_back_to_triage
    ],
    vs_id= "vs_1XyQNuNZAxLScUt4MrmeSvEx",
)

def study_corporate_bond (deal):
    """Use to study corporate bond deal 
    deal can be a deal reference."""

    print("\033[32mStudy corporate bond:\033[0m", deal) # lazy summary
    return "success"

def study_interest_rate ():
    """Use to study interest rate market. 
    """

    print("\033[32mStudy rate market:\033[0m") # lazy summary
    return "success"


def study_structured_product (deal):
    """Use to study a structured product deal 
    deal can be a deal reference."""

    print("\033[32mStudy structured product:\033[0m", deal) # lazy summary
    return "success"

fixed_income_analyst = Agent(
    name="Fixed Income Analyst",
    instructions=(
        "You are a Fixed Income analyst for Blue Hills Research."
        "Your client is a fund portfolio manager who rely on your deep knowledge on all kinds of fixed income instruments."
        "You should help him analyse a specific instrument or a deal, and you should ways carry analytical and critical reasoning."
        "Follow the following routine with the client:"
        "1. First, ask probing questions and understand the what he really like to understand.\n"
        " - unless the he has already provided a reason.\n"
        "2. based on his request, you should decide further to invoke studies of corporate bond,\n"
        "interest rate markets or structured product.\n"
        ""
    ),
    tools=[study_corporate_bond,study_interest_rate,study_structured_product, transfer_back_to_triage],
    vs_id="vs_L7ujcfdkg68ylNZnecprdMbO",
)

def study_macro_economics ():
    """Use to study macro economics. 
    """

    print("\033[32mStudy macro economics:\033[0m") # lazy summary
    return "success"

def study_monetary_policy ():
    """Use to study monetary policy, include Fed study. 
    """

    print("\033[32mStudy monetary policy:\033[0m") # lazy summary
    return "success"

def study_geopolitics ():
    """Use to study geopolitics and implications. 
    """

    print("\033[32mStudy geopolitics issues:\033[0m") # lazy summary
    return "success"


economic_analyst = Agent(
    name="Macro Economic Analyst",
    instructions=(
        "You are a Macro Economic analyst for Blue Hills Research."
        "Your client is a fund portfolio manager who rely on your deep knowledge on economic and geopolitical issues."
        "You should help him analyse a any macro, monetary or fiscal economic questions, and you should ways carry analytical and critical reasoning."
        "Follow the following routine with the client:"
        "1. First, ask probing questions and understand the what he really like to understand.\n"
        " - unless the he has already provided a reason.\n"
        "2. based on his request, you should decide further to invoke studies of macro economics, monetary \n"
        "or geo-political policies.\n"
        ""
    ),
    tools=[study_macro_economics,study_monetary_policy,study_geopolitics, transfer_back_to_triage],
    
)

tax_analyst = Agent(
    name="Tax Specialist",
    instructions=(
        "You are an Tax Specialist from Blue Hills Research."
        "Your client is a fund portfolio manager who rely on your deep knowledge on US and international tax systems."
        "You should help him analyse any tax issues, and you should ways carry analytical and critical reasoning."
        "Follow the following routine with the client:"
        "1. First, ask probing questions and understand the what he really like to understand.\n"
        " - unless the he has already provided a reason.\n"
        "2. based on his request, you should do your best to give advises.\n"
        ""
    ),
    tools=[transfer_back_to_triage],
)



In [ ]:
agent = triage_agent
messages = []

while True:
    user = input("User: ")
    messages.append({"role": "user", "content": user})

    response = run_full_turn(agent, messages)
    agent = response.agent
    messages.extend(response.messages)

User:  Hello


Triage Agent: Hello! I'm an Investment Advisor with Blue Hills Research. How can I assist you today?


User:  help analyse the equity of Startbucks


Triage Agent: transfer_to_equity_analyst({})
Equity Analyst: Sure! What specific aspects of Starbucks equity would you like to analyze? Are you interested in financial performance, management quality, risks, capital allocation strategies, or something else? Let me know your focus so I can best assist you.


User:  let's understand its risks


...

Equity Analyst: I've analyzed the business quality of Starbucks. Here are the key highlights:

1. **Strong Brand Equity**: Starbucks is a well-recognized global brand synonymous with premium coffee and a unique customer experience. Its strong brand helps sustain customer loyalty and pricing power.

2. **Diverse Product Offering**: The company offers a wide range of products, including various coffee beverages, teas, food items, and merchandise. This diversity helps mitigate risks tied to single-product dependence.

3. **Innovative Approach**: Starbucks is known for its innovative offerings, including seasonal beverages, personalized drinks, and expanding food options. The company frequently experiments with new products to keep consumers engaged.

4. **Customer Experience Focus**: The Starbucks experience is designed around creating an inviting atmosphere and high-quality service, which enhances customer loyalty and encourages repeat business.

5. **Global Presence**: Starbucks has an 

User:  can you show me all subsidiaries


Equity Analyst: get_subsidaries({'company_name': 'Starbucks'})
All Starbucks subsidiaries:
             accessionNo                    filedAt     cik ticker  \
0   0000829224-24-000057  2024-11-20T16:08:06-05:00  829224   SBUX   
0   0000829224-24-000057  2024-11-20T16:08:06-05:00  829224   SBUX   
0   0000829224-24-000057  2024-11-20T16:08:06-05:00  829224   SBUX   
0   0000829224-24-000057  2024-11-20T16:08:06-05:00  829224   SBUX   
0   0000829224-24-000057  2024-11-20T16:08:06-05:00  829224   SBUX   
..                   ...                        ...     ...    ...   
25  0000891020-00-002203  2000-12-22T00:00:00-05:00  829224   SBUX   
25  0000891020-00-002203  2000-12-22T00:00:00-05:00  829224   SBUX   
25  0000891020-00-002203  2000-12-22T00:00:00-05:00  829224   SBUX   
25  0000891020-00-002203  2000-12-22T00:00:00-05:00  829224   SBUX   
25  0000891020-00-002203  2000-12-22T00:00:00-05:00  829224   SBUX   

                                              name  \
0             

User:   describe Starbucks management


Equity Analyst: get_exec_comp({'company_name': 'Starbucks'})
                                 id     cik ticker                name  \
0  be026ce93efb16a51d885d3edb809dde  829224   SBUX    Kevin R. Johnson   
1  af9320504bf294f5943445ea27e50cd5  829224   SBUX  Rachel A. Gonzalez   
2  9b52869495c2aebf4b063ea27bbcc74e  829224   SBUX   Laxman Narasimhan   
3  e5598497b6a4579f665d2695712e638b  829224   SBUX         John Culver   
4  cdaf5b5132b797094b9ec496425a8590  829224   SBUX      Michael Conway   

                                            position  year   salary    bonus  \
0       former president and chief executive officer  2022  1302595        0   
1   former executive vice president, general counsel  2022   440232        0   
2                      chief executive officer-elect  2022        0  1600000   
3  former group president and chief operating off...  2022  1000002        0   
4  group president, International and Channel Dev...  2022   924997        0   

   stockAward